In [ ]:
import asf_search as asf
import pandas as pd
import shapely.geometry
import xarray
import sys
sys.path.append('../../')
from spicy_snow.download.sentinel1 import s1_img_search#, download_s1_imgs

import os
os.environ['PROJ_LIB'] = '/Users/zachkeskinen/miniconda3/pkgs/proj-9.1.0-hf909084_1/share/proj'

import pyproj
pyproj.datadir.set_data_dir('/Users/zachkeskinen/miniconda3/pkgs/proj-9.1.0-hf909084_1/share/proj')
pyproj._pyproj_global_context_initialize()


In [ ]:
from shapely.geometry import box
area = box(-114.4, 43, -114.3, 43.1)
dates = ('2019-12-28', '2020-02-02')
# search_result = s1_img_search(area, dates)

#Download from cloud

In [ ]:
from hyp3_sdk.exceptions import AuthenticationError
import hyp3_sdk as sdk
try:
    # .netrc
    hyp3 = sdk.HyP3()
except AuthenticationError:
    # prompt for password
    hyp3 = sdk.HyP3(prompt = True)

In [ ]:
batch = hyp3.find_jobs()

In [ ]:
hyp3.watch(batch)

In [ ]:
granules = search_result['properties.sceneName'].iloc[2:4]

In [ ]:
granules

In [ ]:
import itertools
list2d = [i.job_parameters['granules'] for i in batch.filter_jobs(succeeded = True, failed = False, running = False)]
merged = list(itertools.chain(*list2d))
len(merged)

In [ ]:
rtc_jobs = hyp3.find_jobs(name = 'refresh_test_v3')

In [ ]:
granules = search_result['properties.sceneName'].iloc[2:4]

rtc_jobs = sdk.Batch()
for g in granules:
    # https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_rtc_job
    rtc_jobs += hyp3.submit_rtc_job(g, name = 'cloud-download-test-v2', include_inc_map = True,\
        scale = 'amplitude', dem_matching = False, resolution = 30)

In [ ]:
hyp3._watch_batch(rtc_jobs)

In [ ]:
from tqdm import tqdm
pbar = tqdm(total = len(rtc_jobs))
while not rtc_jobs.complete():
    # to get updated information
    pbar.update(len(rtc_jobs.filter_jobs(succeeded=True, running=False, failed=True)))
    rtc_jobs = hyp3.refresh(rtc_jobs)
pbar.close()

In [ ]:
from tqdm import tqdm
pbar = tqdm(total = len(rtc_jobs))
while not rtc_jobs.complete():
    # to get updated information
    pbar.update(len(rtc_jobs.filter_jobs(succeeded=True, running=False, failed=True)))
    rtc_jobs = hyp3.refresh(rtc_jobs)
pbar.close()
    
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = rtc_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

In [ ]:
u = rtc_jobs[0].files[0]['url']
vv_url = u.replace('.zip', '_VV.tif')
vh_url = u.replace('.zip', '_VH.tif')
inc_url = u.replace('.zip', '_inc_map.tif')

In [ ]:
from urllib.request import urlretrieve
import sys
import time

def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = min(int(count*block_size*100/total_size),100)
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                    (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()
from os.path import basename, exists
def url_download(url, out_fp, overwrite = False):
    if not exists(out_fp) or overwrite == True:
        print(f'Downloading {basename(out_fp)}.')
        urlretrieve(url, out_fp, reporthook)
        print('')

In [ ]:
for i, job in enumerate(rtc_jobs[:]):
    u = job.files[0]['url']
    granule = job.job_parameters['granules'][0]
    print(job.job_parameters)

In [ ]:
import xarray as xr
import rioxarray as rxa
import pandas as pd
das = []
granules = []
for i, job in enumerate(rtc_jobs[:3]):
    u = job.files[0]['url']
    granule = job.job_parameters['granules'][0]
    if granule in granules:
        continue
    granules.append(granule)
    urls = {}
    urls[f'{granule}_VV'] = u.replace('.zip', '_VV.tif')
    urls[f'{granule}_VH'] = u.replace('.zip', '_VH.tif')
    urls[f'{granule}_inc'] = u.replace('.zip', '_inc_map.tif')

    imgs = []
    for j, (name, url) in enumerate(urls.items()):
        url_download(url, f'data/{name}.tif')
        img = rxa.open_rasterio(f'data/{name}.tif')
        img = img.rio.clip([area], 'EPSG:4326')
        img = img.rio.reproject('EPSG:4326')
        band_name = name.replace(f'{granule}_', '')
        img = img.assign_coords(time = pd.to_datetime(granule.split('_')[4]))
        imgs.append(img.assign_coords(band = [band_name]))
    da = xr.concat(imgs, dim = 'band')
    if i != 0:
        da = da.rio.reproject_match(das[0])
    das.append(da)
ds = xr.concat(das, dim = 'time')

In [ ]:
ds = xr.concat([das[0], das[1].rio.reproject_match(das[0]),das[2].rio.reproject_match(das[0])], dim = 'time')

In [ ]:
ds

# Comparing results between sentinel sat and asf_search. Both seem the same

In [ ]:
area = shapely.geometry.box(43, 6, 43.1, 6.1)
dates = ('2018-12-28T00:00:00.000Z', '2020-01-02T00:00:00.000Z')
search_results = asf.geo_search(platform = [asf.PLATFORM.SENTINEL1], intersectsWith = area.wkt,\
        start = dates[0], end = dates[1], processingLevel = asf.PRODUCT_TYPE.GRD_HD)
print(len(search_results))

In [ ]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
dates = ('2018-12-28T00:00:00.000Z', '2020-01-02T00:00:00.000Z')
api = SentinelAPI('zachkeskinen', 'DR1seuss')
footprint = area.wkt
products = api.query(footprint, platformname = 'Sentinel-1', producttype = 'GRD', beginposition = dates,
                     polarisationmode = 'VV+VH', limit=1000)
print(len(products))

In [ ]:
area.bounds[3] < 90
area.bounds[1] > 0
area.bounds[2] < 180
area.bounds[0] > -180

In [ ]:
import asf_search as asf
import pandas as pd
import shapely.geometry
import xarray
import sys
sys.path.append('../../')
from spicy_snow.download.sentinel1 import s1_img_search#, download_s1_imgs

import os
os.environ['PROJ_LIB'] = '/Users/zachkeskinen/miniconda3/pkgs/proj-9.1.0-hf909084_1/share/proj'

import pyproj
pyproj.datadir.set_data_dir('/Users/zachkeskinen/miniconda3/pkgs/proj-9.1.0-hf909084_1/share/proj')
pyproj._pyproj_global_context_initialize()

import rioxarray as rxa
img = rxa.open_rasterio('/Users/zachkeskinen/Documents/spicy-snow/data/fcf.tif', chunk = True)
from shapely.geometry import box
area = box(-114.4, 43, -114.3, 43.1)

In [ ]:
import pickle
import numpy as np
import rioxarray as rxa
import sys
from os.path import expanduser
sys.path.append(expanduser('~/Documents/spicy-snow'))

from spicy_snow.download.sentinel1 import s1_img_search, download_s1_imgs
from spicy_snow.download.forest_cover import download_fcf, add_fcf
from spicy_snow.download.snow_cover import download_snow_cover
with open('/Users/zachkeskinen/Documents/spicy-snow/data/all_ds.pkl', 'rb') as f:
    ds = pickle.load(f)

In [ ]:
ds = download_snow_cover(ds, tmp_dir = '/Users/zachkeskinen/Documents/spicy-snow/data/tmp/', clean = False)

In [ ]:
ds.isel(time = 1)['ims']

In [ ]:
with open('/Users/zachkeskinen/Documents/spicy-snow/data/imsds.pkl', 'rb') as f:
    ims = pickle.dump(f)

In [ ]:
ims = rxa.open_rasterio('/Users/zachkeskinen/Documents/spicy-snow/data/tmp/ims2019281_1km_v1.3.nc', decode_times=False)

In [ ]:
ims = ims.rio.reproject('EPSG:4326').rio.clip_box(-114.4, 43, -114.3, 43.1)

In [ ]:
ds.isel(time = 0)['ims'].plot()

In [ ]:
with open('/Users/zachkeskinen/Documents/spicy-snow/data/s1_dt.pkl', 'rb') as f:
    ds = pickle.load(f)

In [ ]:
ds